In [2]:
from typing import Optional
from dataclasses import dataclass
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

from functools import partial

np.random.seed(42)

In [5]:
dataset = pd.read_parquet("products_with_names.parquet")

In [6]:
dataset.head()

,product_id,name
0,4036767,"Модуль сменный фильтрующий Аквафор КН, 208731"
1,4050873,"Водоочиститель Аквафор модель Кристалл Н, 2059..."
2,4226160,Развиваем мышление (2-3 года) | Земцова Ольга
3,4644911,Lacoste Вода парфюмерная Pour Femme 50 мл
4,4788809,Сменные Кассеты Для Мужской Бритвы Gillette Ma...


In [7]:
dataset.shape

(238443, 2)

In [8]:
documents_dict = {
    doc[1]["product_id"]: doc[1]["name"] for doc in dataset.iterrows()
}

## Подготовим документы

In [9]:
@dataclass
class Document:
    doc_id: int
    name: str


In [10]:
documents = [Document(doc_id=doc[1]["product_id"], name=doc[1]["name"]) for doc in dataset.iterrows()]

In [11]:
documents[:3]

[Document(doc_id=4036767, name='Модуль сменный фильтрующий Аквафор КН, 208731'),
 Document(doc_id=4050873, name='Водоочиститель Аквафор модель Кристалл Н, 205963 //с краном'),
 Document(doc_id=4226160, name='Развиваем мышление (2-3 года) | Земцова Ольга')]

## Метрики качества

### В этом файле собраны взаимодействия пользователей с товарами в поиске
### Будем считать товары, с которыми провзаимодействовали - позитивами, остальные - негативами

In [3]:
query_product_interactions = pd.read_parquet("query_product_interactions.parquet")

In [4]:
query_product_interactions.tail()

,search_query,products
263069,яйчный белок,"[[933874938, 1]]"
263070,якрупа,"[[427186572, 1]]"
263071,янбупели,"[[158178266, 1]]"
263072,японские чистящие средства,"[[762090235, 1], [685325392, 1]]"
263073,яшоды смесь,"[[356053164, 1], [1061471790, 2], [684932688, 1]]"


In [5]:
query_positives = {
    row[1]["search_query"]: set([x[0] for x in row[1]["products"]]) 
    for row in query_product_interactions.iterrows()
}

In [7]:
query_positives["homecat"]

{34629202,
 34629203,
 139864790,
 139864791,
 139864792,
 177668198,
 177668199,
 215201915,
 248843140,
 257675430,
 352414492,
 667810552}

In [15]:
query_positives["молоко"]

{5629622,
 19313381,
 24962122,
 29288925,
 33006465,
 34872818,
 135060571,
 135162769,
 135393420,
 135507645,
 135507650,
 135653559,
 136000122,
 136050530,
 137324023,
 137734030,
 138235117,
 138439744,
 138763861,
 140590409,
 140676405,
 140676428,
 140723110,
 140800364,
 140987736,
 140987738,
 140987743,
 141165025,
 141165033,
 141358850,
 141580513,
 141580514,
 141580515,
 141734954,
 141876199,
 141876200,
 141876201,
 142109013,
 142120311,
 142120312,
 142120332,
 142120577,
 142120579,
 142120587,
 142120588,
 142120589,
 142120596,
 142120791,
 142138049,
 142224881,
 142224890,
 142233202,
 142583958,
 142744764,
 142822562,
 142822563,
 142822574,
 143170479,
 143171016,
 143259875,
 143327779,
 143411505,
 143484939,
 143705652,
 143825149,
 143910224,
 144079776,
 144079777,
 144079778,
 144311416,
 144311417,
 144612073,
 144650933,
 144775459,
 145026870,
 145081636,
 145081637,
 145761824,
 145803938,
 145861337,
 145923184,
 145923190,
 145923192,
 145923205,

In [8]:
good_enough_query_positives = {
    k: v
    for k, v in query_positives.items()
    if len(v) > 10
}

In [9]:
list(good_enough_query_positives.keys())[:100]

['durr сыр',
 'fler alpin',
 'huggies трусики 6',
 'monge для кошек',
 'synergetic мыло',
 'алкогольные напитки',
 'альметте сыр',
 'без глютена хлеб',
 'белковые батончики без сахара',
 'болоньезе',
 'бри сыр',
 'вафли',
 'вегета приправа',
 'веник',
 'вермишель barilla',
 'вино',
 'вискас для кошек сухой',
 'вода святой источник',
 'вода снежинская',
 'вода со вкусом',
 'вода фрутоняня',
 'гель для стрики',
 'голень курицы',
 'грудинка сырокопченая',
 'дистья салата',
 'для бассейна',
 'жидкость для полоскания рта',
 'запеканка',
 'зарядка type с',
 'зубочистки',
 'йогурт клубника',
 'карртошка',
 'каша безмолочная детская fleur alpine',
 'каша молочная агуша',
 'каша молочная детская',
 'каши фрутоняня',
 'кекс с изюмом',
 'кета слабосоленая',
 'кето',
 'кола добрый 2 л',
 'колбаски гриль',
 'конверт подарочный',
 'консервация',
 'конфеиы',
 'копченый сыр',
 'краьовые палочки',
 'ктндер',
 'лего майнкрафт',
 'лимонад добрый',
 'майнкрафт',
 'макроны',
 'масло 82,5 сливочное',
 'масл

In [17]:
validation_queries_set = np.random.choice(
    list(good_enough_query_positives.keys()), size=100, replace=False
).tolist()

In [18]:
validation_queries_set

['нектарин',
 'вода питьевая байкал',
 'помидорка томатная паста',
 'расческа для волос',
 'сыр эмменталь',
 'жилкое мыло',
 'крем детский',
 'массажер',
 'канцелярия',
 'лореаль шампунь',
 'сгущенка рогачевъ',
 'для девочек',
 'иясо',
 'кускус',
 'пешьмени',
 'йогурт густой',
 'кисломолочные напитки',
 'пюре бабушкино лукошко мясное',
 'мягкая игрушка',
 'продукты вегетарианские',
 'грецкий орехи очищенные',
 'безмолочные каши',
 'сок банановый фрутоняня',
 'красный октябрь',
 'матрас надувной двуспальный',
 'оливковый майонез',
 'якобс кофе',
 'куриные котлеты',
 'пылесос вертикальный беспроводной',
 'лоток для кошек',
 'печенье в индивидуальной упаковке',
 'гель для бритья',
 'homecat',
 'брокколи пюре детские',
 'подгузники huggies elite soft',
 'каша детская nestle',
 'подгузники 5 размер',
 'пелёнки',
 'для салата',
 'кольца',
 'творог в пачках',
 'корм для собак мираторг',
 'прокладки ночные',
 'яндекс алиса',
 'рыба',
 'орехи смесь',
 'нарещка',
 'вода сладкая газированная',
 '

In [19]:
validation_query_positives = {
    k: v
    for k, v in query_positives.items()
    if k in validation_queries_set
}

In [20]:
validation_query_positives

{'кускус': {135507645,
  136539247,
  138394207,
  138394211,
  140178670,
  140590406,
  141722913,
  145286245,
  146395520,
  150089774,
  166793425,
  168666402,
  210475106,
  210475117,
  233918326,
  696494950,
  836136193,
  836136198,
  862361538,
  862361545,
  942582758,
  1402033754,
  1614531217},
 'грецкий орехи очищенные': {140030746,
  140174127,
  145861339,
  148268283,
  266606104,
  356292677,
  821658348,
  982219224,
  1247352207,
  1257658902,
  1257701301,
  1257819386,
  1326541838,
  1422549290,
  1467841965,
  1541284595},
 'пудинг ehrmann без сахара': {188564195,
  205790129,
  216966573,
  216966644,
  225399564,
  251823824,
  303850782,
  383709920,
  547560017,
  563419556,
  563419559,
  665863840,
  824550298,
  824550304,
  1052317463,
  1052318858,
  1087488251,
  1302365318,
  1415805047},
 'стейк мираторг говядина': {146439824,
  146439825,
  146439850,
  148809372,
  148809373,
  195592530,
  215437126,
  267891866,
  780485368,
  917664150,
  106

### Точность (precision) и Полнота (recall)

$$ precision = {relevant\_retrieved \over retrieved},\ recall = {relevant\_retrieved \over all\_relevant} $$

In [22]:
@dataclass
class Metrics:
    precision: float
    recall: float
    f1_score: float
        
    def __repr__(self):
        return f"precision = {self.precision}\nrecall = {self.recall}\nf1_score = {self.f1_score}"


In [23]:
def calculate_metrics(ground_truth_set, search_results_set):
    
    # True positives: items that are both in ground truth and search results
    tp = len(ground_truth_set.intersection(search_results_set))
    
    # Precision: tp / (tp + fp)
    precision = tp / len(search_results_set) if len(search_results_set) > 0 else 0.0
    
    # Recall: tp / (tp + fn)
    recall = tp / len(ground_truth_set) if len(ground_truth_set) > 0 else 0.0
    
    # F1-score: harmonic mean of precision and recall
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return Metrics(precision=precision, recall=recall, f1_score=f1_score)


In [25]:
def calculate_validation_metrics(search_function, limit=20):
    metrics = []
    search_results = {}
    for query, positives in validation_query_positives.items():
        search_results[query] = search_function(query=query, limit=limit)
        metrics.append(
            calculate_metrics(positives, [x.doc_id for x in search_results[query]])
        )
    
    return Metrics(
        precision=np.mean([x.precision for x in metrics]),
        recall=np.mean([x.recall for x in metrics]),
        f1_score=np.mean([x.f1_score for x in metrics]),
    )
    

## Случайный поиск

In [26]:
def search_random(documents, query, limit=10):
    return np.random.choice(documents, size=limit).tolist()

In [27]:
random_search_results = search_random(documents, "молоко", limit=20)
random_search_results

[Document(doc_id=148201026, name='Настольная сенсорная светодиодная лампа для школьника, ARTSTYLE TL-305, для офиса, для кабинета'),
 Document(doc_id=298256871, name='Art-Visage Светоотражающий консилер "MIRACLE TOUCH" 101 золотисто-бежевый'),
 Document(doc_id=141540749, name='Крем для лица тональный Чистая Линия Идеальная кожа вв-крем 10в1 Мгновенное преображение с экстрактом розы 40 мл'),
 Document(doc_id=201132147, name='Краска для обуви. одежды, сумки из кожи краситель аэрозоль блеск Черная 190 мл'),
 Document(doc_id=1610391553, name='Каша детская Semper с 5 месяцев безмолочная Кукурузная, сухая, 180 г. Уцененный товар'),
 Document(doc_id=1519559896, name='La Roche-Posay Lipikar Lait Молочко для тела, рук и лица для сухой, очень сухой кожи новорожденных младенцев, детей и взрослых с ниацинамидом и маслом ши для увлажнения, 400 мл. Уцененный товар'),
 Document(doc_id=283274200, name='Пакет майка для фасовки продуктов, бирюзовый, 44х24 см, 100 шт.'),
 Document(doc_id=755331968, name=

In [28]:
calculate_validation_metrics(partial(search_random, documents))

precision = 0.0005
recall = 2.583979328165375e-05
f1_score = 4.914004914004914e-05

## Простейший поиск

In [29]:
def search_dummy(documents, query, limit=10):
    return [doc for doc in documents if query in doc.name][:limit]

[doc for doc in search_dummy(documents, 'яблоко', limit=5)]

[Document(doc_id=141141355, name='Matti мюсли с орехом и яблоком, 250 г'),
 Document(doc_id=147218137, name='Сок Добрый, яблоко персик, 0,2 л х 27'),
 Document(doc_id=147677673, name='Компот детский Агуша, яблоко, черноплодная рябина, клубника, 0,2 л'),
 Document(doc_id=225491326, name='Конфеты Ирис SOLENTO, без глютена, Ассорти двухцветный, яблоко, клубника, манго, арбуз, 250г'),
 Document(doc_id=250287549, name='Сок Сады Придонья Exclusive Зелёное яблоко без сахара, 1 л')]

In [30]:
calculate_validation_metrics(partial(search_dummy, documents), limit=20)

precision = 0.006034090909090909
recall = 0.0032581430007900597
f1_score = 0.0038125982799011595

### Проблемы:
- учет регистра
- пунктуация
- словоформы
- стоп-слова
- опечатки

In [27]:
search_dummy(documents, 'Яблоко', limit=2)

[Document(doc_id=140676424, name='Пюре детское Fleur Alpine Яблоко, с 4 месяцев, 90 г'),
 Document(doc_id=142625167, name='Бабушкино Лукошко Кабачок Яблоко пюре с 5 месяцев, 100 г')]

In [28]:
search_dummy(documents, 'яблоко,', limit=2)

[Document(doc_id=147677673, name='Компот детский Агуша, яблоко, черноплодная рябина, клубника, 0,2 л'),
 Document(doc_id=225491326, name='Конфеты Ирис SOLENTO, без глютена, Ассорти двухцветный, яблоко, клубника, манго, арбуз, 250г')]

In [29]:
search_dummy(documents, 'яблоки', limit=2)

[Document(doc_id=1554768232, name='Российские яблоки Агроном-сад, отборные, 4 шт'),
 Document(doc_id=420836847, name='Набор: капуста брокколи, цветная капуста, тыква, яблоки быстрозамороженные THE LEATLES для приготовления детского питания')]

In [30]:
search_dummy(documents, 'для', limit=2)

[Document(doc_id=4788809, name='Сменные Кассеты Для Мужской Бритвы Gillette Mach3, с 3 лезвиями, прочнее, чем сталь, для точного бритья, 2 шт'),
 Document(doc_id=4789070, name='Gillette Fusion5 мужская бритва, 2 кассеты, с 5 лезвиями, уменьшающими трение, с точным триммером для бороды и усов')]

In [31]:
search_dummy(documents, 'яблако', limit=2)

[]

# Пайплайн представления документов:
1. Обработка текста
2. Индексация

## 1. Обработка текста

In [31]:
sample_documents = list(dataset.loc[:5, "name"])
sample_documents

['Модуль сменный фильтрующий Аквафор КН, 208731',
 'Водоочиститель Аквафор модель Кристалл Н, 205963 //с краном',
 'Развиваем мышление (2-3 года) | Земцова Ольга',
 'Lacoste Вода парфюмерная Pour Femme 50 мл',
 'Сменные Кассеты Для Мужской Бритвы Gillette Mach3, с 3 лезвиями, прочнее, чем сталь, для точного бритья, 2 шт',
 'Gillette Fusion5 мужская бритва, 2 кассеты, с 5 лезвиями, уменьшающими трение, с точным триммером для бороды и усов']

### Этапы
- обработка текста
- лингвистический анализ

## 1.1 Обработка текста:
- приведение к нижнему регистру
- обработка от знаков препинания
- токенизация
- избавление от стоп-слов

### 1.1.1 Приведение к нижнему регистру

In [32]:
def lowercase_text(text: str) -> str:
    return text.lower()

In [33]:
documents_lowercased = [lowercase_text(doc) for doc in sample_documents]
documents_lowercased

['модуль сменный фильтрующий аквафор кн, 208731',
 'водоочиститель аквафор модель кристалл н, 205963 //с краном',
 'развиваем мышление (2-3 года) | земцова ольга',
 'lacoste вода парфюмерная pour femme 50 мл',
 'сменные кассеты для мужской бритвы gillette mach3, с 3 лезвиями, прочнее, чем сталь, для точного бритья, 2 шт',
 'gillette fusion5 мужская бритва, 2 кассеты, с 5 лезвиями, уменьшающими трение, с точным триммером для бороды и усов']

### 1.1.2 Замена символов на эквивалентные

In [34]:
symbols_to_replace = {"ё": "е"}

def replace_symbols(text: str, symbols_to_replace: dict[str, str]) -> str:
    for old, new in symbols_to_replace.items():
        text = text.replace(old, new)
    return text

In [35]:
documents_replaced = [lowercase_text(doc) for doc in documents_lowercased]
documents_replaced

['модуль сменный фильтрующий аквафор кн, 208731',
 'водоочиститель аквафор модель кристалл н, 205963 //с краном',
 'развиваем мышление (2-3 года) | земцова ольга',
 'lacoste вода парфюмерная pour femme 50 мл',
 'сменные кассеты для мужской бритвы gillette mach3, с 3 лезвиями, прочнее, чем сталь, для точного бритья, 2 шт',
 'gillette fusion5 мужская бритва, 2 кассеты, с 5 лезвиями, уменьшающими трение, с точным триммером для бороды и усов']

### 1.1.3 Избавление от знаков препинания

In [36]:
import string

In [37]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [38]:
def process_punctuation_simple(text: str) -> str:
    translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text_without_punc = text.translate(translation_table)
    text_without_double_spaces = ' '.join(text_without_punc.split())
    return text_without_double_spaces

In [39]:
process_punctuation_simple("молоко пастеризованное, 2.5 л")

'молоко пастеризованное 2 5 л'

In [40]:
documents_processed_punct = [
    process_punctuation_simple(doc) for doc in documents_replaced
]
documents_processed_punct

['модуль сменный фильтрующий аквафор кн 208731',
 'водоочиститель аквафор модель кристалл н 205963 с краном',
 'развиваем мышление 2 3 года земцова ольга',
 'lacoste вода парфюмерная pour femme 50 мл',
 'сменные кассеты для мужской бритвы gillette mach3 с 3 лезвиями прочнее чем сталь для точного бритья 2 шт',
 'gillette fusion5 мужская бритва 2 кассеты с 5 лезвиями уменьшающими трение с точным триммером для бороды и усов']

In [41]:
def process_punctuation_smart(text: str) -> str:
    #TODO: придумать более хорошую обработку знаков препинания
    pass

### 1.1.4 Токенизация

In [41]:
def tokenize_simple(text: str) -> list[str]:
    return text.split()

In [42]:
documents_tokenized = [tokenize_simple(doc) for doc in documents_processed_punct]
documents_tokenized

[['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', '208731'],
 ['водоочиститель',
  'аквафор',
  'модель',
  'кристалл',
  'н',
  '205963',
  'с',
  'краном'],
 ['развиваем', 'мышление', '2', '3', 'года', 'земцова', 'ольга'],
 ['lacoste', 'вода', 'парфюмерная', 'pour', 'femme', '50', 'мл'],
 ['сменные',
  'кассеты',
  'для',
  'мужской',
  'бритвы',
  'gillette',
  'mach3',
  'с',
  '3',
  'лезвиями',
  'прочнее',
  'чем',
  'сталь',
  'для',
  'точного',
  'бритья',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужская',
  'бритва',
  '2',
  'кассеты',
  'с',
  '5',
  'лезвиями',
  'уменьшающими',
  'трение',
  'с',
  'точным',
  'триммером',
  'для',
  'бороды',
  'и',
  'усов']]

In [43]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/dandrosov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/dandrosov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [44]:
from nltk.tokenize import word_tokenize

def tokenize_with_punct(text: str) -> list[str]:
    return word_tokenize(text, language="russian")

In [45]:
tokenize_with_punct(documents_replaced[0])

['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', ',', '208731']

### 1.1.5 Удаление стоп-слов

In [46]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dandrosov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
from nltk.corpus import stopwords
russian_stopwords = set(stopwords.words("russian"))

In [48]:
russian_stopwords

{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [49]:
def remove_stopwords_from_doc(doc: list[str], stopwords: set[str]) -> list[str]:
    return [token for token in doc if token not in stopwords]

In [50]:
documents_processed = [remove_stopwords_from_doc(doc, russian_stopwords) for doc in documents_tokenized]
documents_processed

[['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', '208731'],
 ['водоочиститель', 'аквафор', 'модель', 'кристалл', 'н', '205963', 'краном'],
 ['развиваем', 'мышление', '2', '3', 'года', 'земцова', 'ольга'],
 ['lacoste', 'вода', 'парфюмерная', 'pour', 'femme', '50', 'мл'],
 ['сменные',
  'кассеты',
  'мужской',
  'бритвы',
  'gillette',
  'mach3',
  '3',
  'лезвиями',
  'прочнее',
  'сталь',
  'точного',
  'бритья',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужская',
  'бритва',
  '2',
  'кассеты',
  '5',
  'лезвиями',
  'уменьшающими',
  'трение',
  'точным',
  'триммером',
  'бороды',
  'усов']]

## 1.2 Лингвистический анализ

### 1.2.1 Стемминг

#### Для английского языка

In [51]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
text = "The stemmed form of leaves is leaf"
tokens = word_tokenize(text)
stemmed_words = [stemmer.stem(word) for word in tokens]
stemmed_words

['the', 'stem', 'form', 'of', 'leav', 'is', 'leaf']

#### Для русского языка

In [52]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")

def stem(token: str, stemmer: SnowballStemmer) -> str:
    return stemmer.stem(token)


In [54]:
documents_processed

[['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', '208731'],
 ['водоочиститель', 'аквафор', 'модель', 'кристалл', 'н', '205963', 'краном'],
 ['развиваем', 'мышление', '2', '3', 'года', 'земцова', 'ольга'],
 ['lacoste', 'вода', 'парфюмерная', 'pour', 'femme', '50', 'мл'],
 ['сменные',
  'кассеты',
  'мужской',
  'бритвы',
  'gillette',
  'mach3',
  '3',
  'лезвиями',
  'прочнее',
  'сталь',
  'точного',
  'бритья',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужская',
  'бритва',
  '2',
  'кассеты',
  '5',
  'лезвиями',
  'уменьшающими',
  'трение',
  'точным',
  'триммером',
  'бороды',
  'усов']]

In [53]:
documents_stemmed = [[stem(token, stemmer) for token in doc] for doc in documents_processed]
documents_stemmed

[['модул', 'смен', 'фильтр', 'аквафор', 'кн', '208731'],
 ['водоочистител', 'аквафор', 'модел', 'кристалл', 'н', '205963', 'кран'],
 ['развива', 'мышлен', '2', '3', 'год', 'земцов', 'ольг'],
 ['lacoste', 'вод', 'парфюмерн', 'pour', 'femme', '50', 'мл'],
 ['смен',
  'кассет',
  'мужск',
  'бритв',
  'gillette',
  'mach3',
  '3',
  'лезв',
  'прочн',
  'стал',
  'точн',
  'брит',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужск',
  'бритв',
  '2',
  'кассет',
  '5',
  'лезв',
  'уменьша',
  'трен',
  'точн',
  'триммер',
  'бород',
  'ус']]

### 1.2.2 Лемматизация

#### Для английского языка

In [55]:
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/dandrosov/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dandrosov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [57]:
text = "The lemmatized form of leaves is leaf"
tokens = word_tokenize(text)
lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
print(lemmatized_words)

['The', 'lemmatized', 'form', 'of', 'leaf', 'is', 'leaf']


#### Для русского языка

In [58]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [59]:
morph.parse("воды")

[Parse(word='воды', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='вода', score=0.87619, methods_stack=((DictionaryAnalyzer(), 'воды', 55, 1),)),
 Parse(word='воды', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='вода', score=0.061904, methods_stack=((DictionaryAnalyzer(), 'воды', 55, 10),)),
 Parse(word='воды', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='вода', score=0.05238, methods_stack=((DictionaryAnalyzer(), 'воды', 55, 7),)),
 Parse(word='воды', tag=OpencorporaTag('NOUN,inan,masc plur,nomn'), normal_form='вод', score=0.004761, methods_stack=((DictionaryAnalyzer(), 'воды', 34, 6),)),
 Parse(word='воды', tag=OpencorporaTag('NOUN,inan,masc plur,accs'), normal_form='вод', score=0.004761, methods_stack=((DictionaryAnalyzer(), 'воды', 34, 9),))]

In [60]:
def lemmatize_token(token: str, lemmatizer: MorphAnalyzer) -> str:
    return lemmatizer.normal_forms(token)[0]

def lemmatize_document(doc: list[str], lemmatizer: MorphAnalyzer) -> list[str]:
    return [lemmatize_token(token, lemmatizer) for token in doc]

In [62]:
documents_processed

[['модуль', 'сменный', 'фильтрующий', 'аквафор', 'кн', '208731'],
 ['водоочиститель', 'аквафор', 'модель', 'кристалл', 'н', '205963', 'краном'],
 ['развиваем', 'мышление', '2', '3', 'года', 'земцова', 'ольга'],
 ['lacoste', 'вода', 'парфюмерная', 'pour', 'femme', '50', 'мл'],
 ['сменные',
  'кассеты',
  'мужской',
  'бритвы',
  'gillette',
  'mach3',
  '3',
  'лезвиями',
  'прочнее',
  'сталь',
  'точного',
  'бритья',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужская',
  'бритва',
  '2',
  'кассеты',
  '5',
  'лезвиями',
  'уменьшающими',
  'трение',
  'точным',
  'триммером',
  'бороды',
  'усов']]

In [61]:
documents_lemmatized = [lemmatize_document(doc, morph) for doc in documents_processed]
documents_lemmatized

[['модуль', 'сменный', 'фильтровать', 'аквафора', 'кн', '208731'],
 ['водоочиститель', 'аквафора', 'модель', 'кристалл', 'н', '205963', 'кран'],
 ['развивать', 'мышление', '2', '3', 'год', 'земцов', 'ольга'],
 ['lacoste', 'вода', 'парфюмерный', 'pour', 'femme', '50', 'мл'],
 ['сменный',
  'кассета',
  'мужской',
  'бритва',
  'gillette',
  'mach3',
  '3',
  'лезвие',
  'прочный',
  'сталь',
  'точный',
  'бритьё',
  '2',
  'шт'],
 ['gillette',
  'fusion5',
  'мужской',
  'бритва',
  '2',
  'кассета',
  '5',
  'лезвие',
  'уменьшать',
  'трение',
  'точный',
  'триммер',
  'борода',
  'усов']]

## 1.3 Собираем всё вместе

In [63]:
def process_document(
    doc: str,
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer,
) -> Document:
    doc = lowercase_text(doc)
    doc = replace_symbols(doc, symbols_to_replace)
    doc = process_punctuation_simple(doc)
    doc_tokens = tokenize_simple(doc)
    doc_tokens = remove_stopwords_from_doc(doc_tokens, russian_stopwords)
    return lemmatize_document(doc_tokens, lemmatizer)

def process_documents(
    docs: list[str],
    symbols_to_replace: dict[str, str], 
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer,
    progress_bar: bool = True
) -> list[tuple[int, str]]:
    iterator = docs
    if progress_bar:
        iterator = tqdm(docs)
    
    return [
        (doc.doc_id, process_document(doc.name, symbols_to_replace, russian_stopwords, lemmatizer))
        for doc in iterator
    ]


In [64]:
documents[1000]

Document(doc_id=166282743, name='Кухонная вытяжка Weissgauff Aura 850 BL, черный')

In [65]:
process_document(documents[1000].name, symbols_to_replace, russian_stopwords, morph)

['кухонный', 'вытяжка', 'weissgauff', 'aura', '850', 'bl', 'чёрный']

In [66]:
documents_processed = process_documents(documents, symbols_to_replace, russian_stopwords, morph)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 238443/238443 [02:25<00:00, 1644.10it/s]


  6%|▋         | 15366/238443 [00:20<04:14, 875.63it/s]






  6%|▋         | 15455/238443 [00:20<04:14, 876.67it/s]






  7%|▋         | 15543/238443 [00:21<04:17, 867.10it/s]






  7%|▋         | 15656/238443 [00:21<03:55, 944.59it/s]






  7%|▋         | 15751/238443 [00:21<04:07, 901.21it/s]






  7%|▋         | 15842/238443 [00:21<04:16, 866.28it/s]






  7%|▋         | 15930/238443 [00:21<04:28, 829.45it/s]






  7%|▋         | 16014/238443 [00:21<04:43, 785.91it/s]






  7%|▋         | 16094/238443 [00:21<04:44, 782.01it/s]






  7%|▋         | 16173/238443 [00:21<04:49, 766.81it/s]






  7%|▋         | 16254/238443 [00:21<04:46, 776.62it/s]






  7%|▋         | 16337/238443 [00:22<04:40, 791.74it/s]






  7%|▋         | 16417/238443 [00:22<04:59, 740.49it/s]






  7%|▋         | 16504/238443 [00:22<04:46, 774.64it/s]






  7%|▋         | 16583/238443 [00:22<04:53, 756.26it/s]






  7%|▋         | 16660/238443 [00:22<04:59, 741.54it/s]






  7%|▋  

 12%|█▏        | 29762/238443 [00:41<04:36, 753.87it/s]






 13%|█▎        | 29843/238443 [00:41<04:30, 770.24it/s]






 13%|█▎        | 29921/238443 [00:41<04:44, 732.10it/s]






 13%|█▎        | 30003/238443 [00:41<04:36, 754.76it/s]






 13%|█▎        | 30079/238443 [00:41<04:37, 751.83it/s]






 13%|█▎        | 30155/238443 [00:42<04:37, 750.34it/s]






 13%|█▎        | 30231/238443 [00:42<04:58, 696.92it/s]






 13%|█▎        | 30304/238443 [00:42<04:55, 703.67it/s]






 13%|█▎        | 30378/238443 [00:42<04:52, 712.21it/s]






 13%|█▎        | 30450/238443 [00:42<04:58, 696.79it/s]






 13%|█▎        | 30530/238443 [00:42<04:47, 721.98it/s]






 13%|█▎        | 30606/238443 [00:42<04:44, 730.56it/s]






 13%|█▎        | 30689/238443 [00:42<04:33, 758.36it/s]






 13%|█▎        | 30766/238443 [00:42<04:33, 758.83it/s]






 13%|█▎        | 30843/238443 [00:43<04:34, 756.13it/s]






 13%|█▎        | 30922/238443 [00:43<04:31, 765.31it/s]






 13%|█▎ 

 19%|█▊        | 44668/238443 [01:02<04:12, 766.09it/s]






 19%|█▉        | 44756/238443 [01:02<04:03, 795.06it/s]






 19%|█▉        | 44840/238443 [01:02<04:00, 803.37it/s]






 19%|█▉        | 44924/238443 [01:02<03:58, 812.99it/s]






 19%|█▉        | 45006/238443 [01:02<03:57, 813.32it/s]






 19%|█▉        | 45091/238443 [01:02<03:54, 823.24it/s]






 19%|█▉        | 45174/238443 [01:02<03:54, 822.96it/s]






 19%|█▉        | 45257/238443 [01:02<03:55, 818.68it/s]






 19%|█▉        | 45343/238443 [01:02<03:52, 829.20it/s]






 19%|█▉        | 45426/238443 [01:02<04:01, 797.70it/s]






 19%|█▉        | 45507/238443 [01:03<04:06, 781.73it/s]






 19%|█▉        | 45586/238443 [01:03<04:12, 764.66it/s]






 19%|█▉        | 45663/238443 [01:03<04:16, 751.51it/s]






 19%|█▉        | 45739/238443 [01:03<04:26, 721.92it/s]






 19%|█▉        | 45816/238443 [01:03<04:22, 734.58it/s]






 19%|█▉        | 45892/238443 [01:03<04:21, 735.80it/s]






 19%|█▉ 

 25%|██▍       | 58865/238443 [01:22<04:04, 734.75it/s]






 25%|██▍       | 58939/238443 [01:23<04:08, 723.22it/s]






 25%|██▍       | 59012/238443 [01:23<04:08, 720.85it/s]






 25%|██▍       | 59088/238443 [01:23<04:04, 732.35it/s]






 25%|██▍       | 59163/238443 [01:23<04:04, 732.00it/s]






 25%|██▍       | 59237/238443 [01:23<04:16, 698.18it/s]






 25%|██▍       | 59321/238443 [01:23<04:02, 738.78it/s]






 25%|██▍       | 59396/238443 [01:23<04:05, 728.86it/s]






 25%|██▍       | 59470/238443 [01:23<04:06, 725.54it/s]






 25%|██▍       | 59543/238443 [01:23<04:11, 712.47it/s]






 25%|██▌       | 59631/238443 [01:23<03:55, 758.60it/s]






 25%|██▌       | 59708/238443 [01:24<03:59, 746.07it/s]






 25%|██▌       | 59793/238443 [01:24<03:50, 774.17it/s]






 25%|██▌       | 59873/238443 [01:24<03:48, 780.96it/s]






 25%|██▌       | 59955/238443 [01:24<03:46, 789.24it/s]






 25%|██▌       | 60035/238443 [01:24<03:50, 774.31it/s]






 25%|██▌

...

 74%|███████▍  | 176116/238443 [04:06<01:19, 784.68it/s]






 74%|███████▍  | 176211/238443 [04:06<01:14, 831.73it/s]






 74%|███████▍  | 176318/238443 [04:06<01:09, 899.61it/s]






 74%|███████▍  | 176409/238443 [04:06<01:11, 867.43it/s]






 74%|███████▍  | 176499/238443 [04:06<01:10, 876.02it/s]






 74%|███████▍  | 176587/238443 [04:07<01:15, 814.42it/s]






 74%|███████▍  | 176677/238443 [04:07<01:13, 837.91it/s]






 74%|███████▍  | 176762/238443 [04:07<01:16, 809.10it/s]






 74%|███████▍  | 176844/238443 [04:07<01:16, 803.44it/s]






 74%|███████▍  | 176925/238443 [04:07<01:18, 783.32it/s]






 74%|███████▍  | 177007/238443 [04:07<01:17, 792.21it/s]






 74%|███████▍  | 177087/238443 [04:07<01:22, 746.28it/s]






 74%|███████▍  | 177163/238443 [04:07<01:29, 686.39it/s]






 74%|███████▍  | 177233/238443 [04:07<01:31, 667.81it/s]






 74%|███████▍  | 177301/238443 [04:08<01:31, 670.22it/s]






 74%|███████▍  | 177369/238443 [04:08<01:32, 659.91it/s

 80%|███████▉  | 190368/238443 [04:26<01:06, 726.18it/s]






 80%|███████▉  | 190446/238443 [04:27<01:04, 740.80it/s]






 80%|███████▉  | 190526/238443 [04:27<01:03, 757.42it/s]






 80%|███████▉  | 190609/238443 [04:27<01:01, 778.43it/s]






 80%|███████▉  | 190687/238443 [04:27<01:03, 751.14it/s]






 80%|████████  | 190767/238443 [04:27<01:02, 764.63it/s]






 80%|████████  | 190847/238443 [04:27<01:01, 771.89it/s]






 80%|████████  | 190929/238443 [04:27<01:00, 785.53it/s]






 80%|████████  | 191015/238443 [04:27<00:59, 802.97it/s]






 80%|████████  | 191096/238443 [04:27<01:01, 775.61it/s]






 80%|████████  | 191175/238443 [04:28<01:00, 778.42it/s]






 80%|████████  | 191264/238443 [04:28<00:58, 807.72it/s]






 80%|████████  | 191345/238443 [04:28<00:59, 796.44it/s]






 80%|████████  | 191425/238443 [04:28<00:59, 791.29it/s]






 80%|████████  | 191505/238443 [04:28<00:59, 783.61it/s]






 80%|████████  | 191584/238443 [04:28<01:00, 769.60it/s

 86%|████████▌ | 205327/238443 [04:47<00:37, 886.74it/s]






 86%|████████▌ | 205416/238443 [04:47<00:37, 875.06it/s]






 86%|████████▌ | 205505/238443 [04:47<00:37, 878.85it/s]






 86%|████████▌ | 205593/238443 [04:47<00:38, 854.41it/s]






 86%|████████▋ | 205679/238443 [04:47<00:39, 821.03it/s]






 86%|████████▋ | 205762/238443 [04:47<00:42, 767.43it/s]






 86%|████████▋ | 205840/238443 [04:47<00:42, 764.44it/s]






 86%|████████▋ | 205917/238443 [04:47<00:43, 755.20it/s]






 86%|████████▋ | 205995/238443 [04:48<00:42, 761.99it/s]






 86%|████████▋ | 206072/238443 [04:48<00:42, 758.57it/s]






 86%|████████▋ | 206149/238443 [04:48<00:43, 747.91it/s]






 86%|████████▋ | 206224/238443 [04:48<00:45, 714.41it/s]






 87%|████████▋ | 206296/238443 [04:48<00:46, 688.72it/s]






 87%|████████▋ | 206370/238443 [04:48<00:45, 700.83it/s]






 87%|████████▋ | 206441/238443 [04:48<00:46, 693.48it/s]






 87%|████████▋ | 206516/238443 [04:48<00:45, 707.18it/s

 92%|█████████▏| 219552/238443 [05:07<00:25, 733.76it/s]






 92%|█████████▏| 219627/238443 [05:07<00:26, 721.37it/s]






 92%|█████████▏| 219713/238443 [05:07<00:24, 759.47it/s]






 92%|█████████▏| 219795/238443 [05:07<00:24, 774.98it/s]






 92%|█████████▏| 219874/238443 [05:08<00:25, 741.84it/s]






 92%|█████████▏| 219949/238443 [05:08<00:25, 727.62it/s]






 92%|█████████▏| 220024/238443 [05:08<00:25, 733.40it/s]






 92%|█████████▏| 220098/238443 [05:08<00:24, 734.16it/s]






 92%|█████████▏| 220178/238443 [05:08<00:24, 752.90it/s]






 92%|█████████▏| 220254/238443 [05:08<00:25, 704.98it/s]






 92%|█████████▏| 220330/238443 [05:08<00:25, 719.65it/s]






 92%|█████████▏| 220416/238443 [05:08<00:23, 759.16it/s]






 92%|█████████▏| 220499/238443 [05:08<00:23, 779.42it/s]






 93%|█████████▎| 220583/238443 [05:08<00:22, 796.70it/s]






 93%|█████████▎| 220664/238443 [05:09<00:22, 790.46it/s]






 93%|█████████▎| 220746/238443 [05:09<00:22, 798.68it/s

 98%|█████████▊| 234490/238443 [05:27<00:04, 918.77it/s]






 98%|█████████▊| 234583/238443 [05:28<00:04, 889.45it/s]






 98%|█████████▊| 234681/238443 [05:28<00:04, 912.70it/s]






 98%|█████████▊| 234773/238443 [05:28<00:04, 850.78it/s]






 98%|█████████▊| 234860/238443 [05:28<00:04, 814.79it/s]






 99%|█████████▊| 234943/238443 [05:28<00:04, 789.34it/s]






 99%|█████████▊| 235023/238443 [05:28<00:04, 788.30it/s]






 99%|█████████▊| 235103/238443 [05:28<00:04, 776.27it/s]






 99%|█████████▊| 235187/238443 [05:28<00:04, 790.57it/s]






 99%|█████████▊| 235267/238443 [05:28<00:04, 788.26it/s]






 99%|█████████▊| 235347/238443 [05:29<00:04, 768.16it/s]






 99%|█████████▊| 235425/238443 [05:29<00:04, 750.89it/s]






 99%|█████████▉| 235501/238443 [05:29<00:03, 744.49it/s]






 99%|█████████▉| 235576/238443 [05:29<00:03, 728.25it/s]






 99%|█████████▉| 235649/238443 [05:29<00:03, 709.52it/s]






 99%|█████████▉| 235721/238443 [05:29<00:03, 695.19it/s

In [67]:
with open("documents_processed.json", "w") as f:
    json.dump(documents_processed, f)


In [68]:
with open("documents_processed.json", "r") as f:
    documents_processed = json.load(f)

In [69]:
documents_processed[0]

[4036767, ['модуль', 'сменный', 'фильтровать', 'аквафора', 'кн', '208731']]

## 2. Формирование индекса

In [71]:
sample_documents_processed = documents_processed[:10]

In [72]:
corpus_tokens = [
    (token, doc_id) for doc_id, doc_tokens in sample_documents_processed for token in doc_tokens
]
corpus_tokens

[('модуль', 4036767),
 ('сменный', 4036767),
 ('фильтровать', 4036767),
 ('аквафора', 4036767),
 ('кн', 4036767),
 ('208731', 4036767),
 ('водоочиститель', 4050873),
 ('аквафора', 4050873),
 ('модель', 4050873),
 ('кристалл', 4050873),
 ('н', 4050873),
 ('205963', 4050873),
 ('кран', 4050873),
 ('развивать', 4226160),
 ('мышление', 4226160),
 ('2', 4226160),
 ('3', 4226160),
 ('год', 4226160),
 ('земцов', 4226160),
 ('ольга', 4226160),
 ('lacoste', 4644911),
 ('вода', 4644911),
 ('парфюмерный', 4644911),
 ('pour', 4644911),
 ('femme', 4644911),
 ('50', 4644911),
 ('мл', 4644911),
 ('сменный', 4788809),
 ('кассета', 4788809),
 ('мужской', 4788809),
 ('бритва', 4788809),
 ('gillette', 4788809),
 ('mach3', 4788809),
 ('3', 4788809),
 ('лезвие', 4788809),
 ('прочный', 4788809),
 ('сталь', 4788809),
 ('точный', 4788809),
 ('бритьё', 4788809),
 ('2', 4788809),
 ('шт', 4788809),
 ('gillette', 4789070),
 ('fusion5', 4789070),
 ('мужской', 4789070),
 ('бритва', 4789070),
 ('2', 4789070),
 ('кас

In [73]:
corpus_tokens.sort()

In [74]:
corpus_tokens

[('110', 5543514),
 ('12', 5501348),
 ('2', 4226160),
 ('2', 4788809),
 ('2', 4789070),
 ('2', 5629579),
 ('205963', 4050873),
 ('208731', 4036767),
 ('3', 4226160),
 ('3', 4788809),
 ('5', 4789070),
 ('50', 4644911),
 ('action', 5629579),
 ('bourjois', 5501348),
 ('femme', 4644911),
 ('fusion5', 4789070),
 ('gillette', 4788809),
 ('gillette', 4789070),
 ('glamour', 5501348),
 ('lacoste', 4644911),
 ('lady', 5176579),
 ('mach3', 4788809),
 ('oxi', 5629579),
 ('pour', 4644911),
 ('speed', 5176579),
 ('stick', 5176579),
 ('vanish', 5629579),
 ('volume', 5501348),
 ('wella', 5543514),
 ('аквафора', 4036767),
 ('аквафора', 4050873),
 ('бельё', 5629579),
 ('борода', 4789070),
 ('бритва', 4788809),
 ('бритва', 4789070),
 ('бритьё', 4788809),
 ('ваниш', 5629579),
 ('вода', 4644911),
 ('водоочиститель', 4050873),
 ('волос', 5543514),
 ('год', 4226160),
 ('дезодорант', 5176579),
 ('жидкий', 5629579),
 ('земцов', 4226160),
 ('кассета', 4788809),
 ('кассета', 4789070),
 ('кислородный', 5629579),


In [75]:
index = {}

for token, doc_id in corpus_tokens:
    if token in index:
        if doc_id == index[token][-1]:
            continue
        index[token].append(doc_id)
    else:
        index[token] = [doc_id]


In [76]:
index

{'110': [5543514],
 '12': [5501348],
 '2': [4226160, 4788809, 4789070, 5629579],
 '205963': [4050873],
 '208731': [4036767],
 '3': [4226160, 4788809],
 '5': [4789070],
 '50': [4644911],
 'action': [5629579],
 'bourjois': [5501348],
 'femme': [4644911],
 'fusion5': [4789070],
 'gillette': [4788809, 4789070],
 'glamour': [5501348],
 'lacoste': [4644911],
 'lady': [5176579],
 'mach3': [4788809],
 'oxi': [5629579],
 'pour': [4644911],
 'speed': [5176579],
 'stick': [5176579],
 'vanish': [5629579],
 'volume': [5501348],
 'wella': [5543514],
 'аквафора': [4036767, 4050873],
 'бельё': [5629579],
 'борода': [4789070],
 'бритва': [4788809, 4789070],
 'бритьё': [4788809],
 'ваниш': [5629579],
 'вода': [4644911],
 'водоочиститель': [4050873],
 'волос': [5543514],
 'год': [4226160],
 'дезодорант': [5176579],
 'жидкий': [5629579],
 'земцов': [4226160],
 'кассета': [4788809, 4789070],
 'кислородный': [5629579],
 'кн': [4036767],
 'кран': [4050873],
 'краска': [5543514],
 'кристалл': [4050873],
 'л':

In [77]:
def create_index(documents_processed: list[list[str]], limit: Optional[int] = None):
    sample_documents_processed = documents_processed[:limit]
    corpus_tokens = [
        (token, doc_id) for doc_id, doc_tokens in sample_documents_processed for token in doc_tokens
    ]
    corpus_tokens.sort()
    
    index = {}

    for token, doc_id in corpus_tokens:
        if token in index:
            if doc_id == index[token][-1]:
                continue
            index[token].append(doc_id)
        else:
            index[token] = [doc_id]
    
    return index


In [78]:
documents_index = create_index(documents_processed)

In [79]:
with open("documents_index.json", "w") as f:
    json.dump(documents_index, f)


In [80]:
with open("documents_index.json", "r") as f:
    documents_index = json.load(f)


## 3. Поиск по обратному индексу

In [83]:
def search_single_token_query(
    query: str, 
    index: dict[str, list[int]], 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer, 
    limit: Optional[int] = None,
) -> list[str]:
    query = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)[0]
    return [(result_doc_id, documents_dict[result_doc_id]) for result_doc_id in index[query]][:limit]

In [84]:
search_single_token_query(
    "диван",
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    10,
)

[(28876761,
  'Держатель пялец для вышивания на диване, кресле или за столом, Hobby Nurge'),
 (169810294,
  'Наматрасник 160х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 (169810305,
  'Наматрасник 80х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 (169810314,
  'Наматрасник 90х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 (170627411,
  'Наматрасник 180х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 (170627453,
  'Наматрасник 140х200 непромокаемый с р

### Максимизируем точность

In [85]:
def merge_and(posting_lists):
    if not posting_lists:
        return []
    
    posting_lists.sort(key=len)
    result = posting_lists[0]
    
    for i in range(1, len(posting_lists)):
        current_list = posting_lists[i]
        new_result = []
        ptr1, ptr2 = 0, 0
        
        while ptr1 < len(result) and ptr2 < len(current_list):
            if result[ptr1] == current_list[ptr2]:
                new_result.append(result[ptr1])
                ptr1 += 1
                ptr2 += 1
            elif result[ptr1] < current_list[ptr2]:
                ptr1 += 1
            else:
                ptr2 += 1
        
        result = new_result
        if not result:
            break
    
    return result

In [86]:
def search_with_and_condition(
    index: dict[str, list[int]], 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer,
    query: str, 
    limit: Optional[int] = None,
) -> list[str]:
    query_processed = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)
    if not query_processed:
        return []
    merged_posting_lists = merge_and(
        [index.get(token, []) for token in query_processed]
    )
    
    return [
        Document(doc_id=result_doc_id, name=documents_dict[result_doc_id]) for result_doc_id in merged_posting_lists
    ][:limit]

In [87]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "диван кровать",
    10,
)

[Document(doc_id=177310072, name='Плед Павлайн Ирен какао 150х200 , плед на кровать; плед на диван; покрывало на кровать; покрывало на диван; плед 150 на 200; плед 200 на 220; 150*200; плед 200х220; плед 150х200; полуторка'),
 Document(doc_id=229424677, name='Пылесос для дома Jimmy WB55, ручной, для мебели и для удаления пылевых клещей, от шерсти животных, с контейнером, для диванов и кроватей, ультрафиолетовая и ультразвуковая чистка, мощный 600Вт'),
 Document(doc_id=229425205, name='Пылесос для дома Jimmy JV35, ручной, для мебели, для удаления пылевых клещей, от шерсти животных, с контейнером, для диванов и кроватей, ультрафиолетовая чистка и обработка горячим воздухом'),
 Document(doc_id=245377779, name='Покрывало на диван/кровать MIX 140х210 см'),
 Document(doc_id=255343578, name='Покрывало "Сладкий Сон" 220х240 см, двустороннее, однотонное, стеганое Ультрастеп. На кровать, на диван.'),
 Document(doc_id=262376994, name='Наматрасник стеганый 160 200 водонепроницаемый непромокаемый М

In [88]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "двухъярусная кровать",
    10,
)

[Document(doc_id=355105396, name='Двухъярусная кровать для кукол 46 см Our genration'),
 Document(doc_id=355115836, name="Набор мебели игровой Li'l Woodzeez Детская и двухъярусная кровать")]

In [89]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное",
    10,
)

[Document(doc_id=140987734, name='Valio Молоко Ультрапастеризованное 1.5% 250мл. 1шт.'),
 Document(doc_id=140987737, name='Valio Молоко Ультрапастеризованное 2.5% 1000мл. 1шт.'),
 Document(doc_id=140987742, name='Valio Молоко Ультрапастеризованное 3.5% 1000мл. 1шт.'),
 Document(doc_id=141165025, name='Молоко ультрапастеризованное Простоквашино, 3,2%, 950 мл'),
 Document(doc_id=141270536, name='Молоко Вкуснотеево ультрапастеризованное, 2,5%, 950 мл'),
 Document(doc_id=141270537, name='Молоко ультрапастеризованное, 3,2%, 950 мл, Вкуснотеево'),
 Document(doc_id=141580513, name='Молоко ультрапастеризованное 3,5%, 1 л, Parmalat'),
 Document(doc_id=141580514, name='Молоко ультрапастеризованное 1,8% 1 л, Parmalat'),
 Document(doc_id=141580515, name='Parmalat Молоко Ультрапастеризованное 0.5% 1000мл. 1шт.'),
 Document(doc_id=141580516, name='Parmalat Молоко Ультрапастеризованное 3.5% 200мл. 1шт.')]

In [90]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное 1 л",
    10,
)

[Document(doc_id=141580513, name='Молоко ультрапастеризованное 3,5%, 1 л, Parmalat'),
 Document(doc_id=141580514, name='Молоко ультрапастеризованное 1,8% 1 л, Parmalat'),
 Document(doc_id=141580518, name='Молоко Белый Город ультрапастеризованное 3,2%, 1 л'),
 Document(doc_id=141876199, name='Молоко ультрапастеризованное 3,5%, 12 шт х 1 л, Parmalat'),
 Document(doc_id=141876200, name='Молоко ультрапастеризованное 1,8% 12 шт х 1 л, Parmalat'),
 Document(doc_id=141876201, name='Молоко Parmalat ультрапастеризованное 0,5%, 12 шт х 1 л'),
 Document(doc_id=143412236, name='Arla Natura Молоко безлактозное, ультрапастеризованное, 1,5%, 1 л'),
 Document(doc_id=156418189, name='Молоко ультрапастеризованное 3,5%, 12 шт х 1 л, Parmalat'),
 Document(doc_id=156418190, name='Молоко ультрапастеризованное 1,8% 12 шт х 1 л, Parmalat'),
 Document(doc_id=156418192, name='Молоко ультрапастеризованное 3,5%, 1 л, Parmalat')]

In [91]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "для",
    10,
)

[]

In [92]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "средство для мытья посуды",
    10,
)

[Document(doc_id=6106583, name='Средство для мытья посуды "Frosch Лимон", 500 мл'),
 Document(doc_id=6106584, name='Средство для мытья посуды "Frosch", с ароматом лимона, 1 л'),
 Document(doc_id=23772174, name='LION Chamgreen Средство для мытья посуды, овощей и фруктов Зеленый чай 1200 мл'),
 Document(doc_id=26431422, name='Средство для мытья посуды, овощей и фруктов BioMio Bio-Care, гипоаллергенное, экологичное, без запаха, 450 мл'),
 Document(doc_id=26431424, name='Средство для мытья посуды, овощей и\xa0фруктов BioMio Bio-Care, гипоаллергенное, экологичное, с\xa0эфирным маслом мяты, 450\xa0мл'),
 Document(doc_id=26431428, name='Средство для мытья посуды, овощей и фруктов BioMio Bio-Care, гипоаллергенное, экологичное, с эфирным маслом мандарина, 450 мл'),
 Document(doc_id=29287948, name='LION Chamgreen Средство для мытья посуды, овощей и\xa0фруктов Японский абрикос, мягкая упаковка, 960\xa0мл'),
 Document(doc_id=33319465, name='Средство для мытья посуды SYNERGETIC антибактериальное,\x

In [93]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "скатерть на стол",
    10,
)

[Document(doc_id=148827514, name='Скатерть на стол 145х180 см из хлопка водоотталкивающая, скатерть кухонная на стол с пропиткой для праздника кухни дома сада и дачи пятнозащитная кухонный декор в подарок, белая с розами прованс'),
 Document(doc_id=148827515, name='Скатерть на стол Fresca Design 145х220 см'),
 Document(doc_id=149750466, name='Скатерть круглая 185 см на стол из хлопка водоотталкивающая, скатерть на круглый стол кухонная с пропиткой от воды для праздника кухни дома сада и дачи пятнозащитная кухонный текстиль декор в подарок'),
 Document(doc_id=151998971, name='Скатерть на стол UNTERZO home прямоугольная 145х180 см хлопок'),
 Document(doc_id=155432339, name='Скатерть на стол круглая Fresca Design 185х185 см хлопок'),
 Document(doc_id=155523748, name='Скатерть на стол овальная 160х220 см хлопок не боится пятен'),
 Document(doc_id=155523854, name='Скатерть на стол Fresca Design 145х220 см'),
 Document(doc_id=155523861, name='Скатерть на стол квадратная 145 см хлопок'),
 Doc

In [94]:
search_with_and_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "скатерть для стола",
    10,
)

[Document(doc_id=148827514, name='Скатерть на стол 145х180 см из хлопка водоотталкивающая, скатерть кухонная на стол с пропиткой для праздника кухни дома сада и дачи пятнозащитная кухонный декор в подарок, белая с розами прованс'),
 Document(doc_id=148827515, name='Скатерть на стол Fresca Design 145х220 см'),
 Document(doc_id=149750466, name='Скатерть круглая 185 см на стол из хлопка водоотталкивающая, скатерть на круглый стол кухонная с пропиткой от воды для праздника кухни дома сада и дачи пятнозащитная кухонный текстиль декор в подарок'),
 Document(doc_id=151998971, name='Скатерть на стол UNTERZO home прямоугольная 145х180 см хлопок'),
 Document(doc_id=155432339, name='Скатерть на стол круглая Fresca Design 185х185 см хлопок'),
 Document(doc_id=155523748, name='Скатерть на стол овальная 160х220 см хлопок не боится пятен'),
 Document(doc_id=155523854, name='Скатерть на стол Fresca Design 145х220 см'),
 Document(doc_id=155523861, name='Скатерть на стол квадратная 145 см хлопок'),
 Doc

In [95]:
calculate_validation_metrics(
    partial(
        search_with_and_condition,
        documents_index,
        documents_dict,
        symbols_to_replace,
        russian_stopwords,
        morph,
    ),
    limit=20
)

precision = 0.18578044178044176
recall = 0.1411223352381426
f1_score = 0.14620057935041111

### Максимизируем полноту

In [96]:
def merge_or(posting_lists):
    if not posting_lists:
        return []
    
    result = set()
    
    for lst in posting_lists:
        result.update(lst)
    
    return sorted(result)

In [97]:
def search_with_or_condition(
    index: dict[str, list[int]], 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer, 
    query: str, 
    limit: Optional[int] = None,
) -> list[str]:
    query_processed = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)
    if not query_processed:
        return []
    merged_posting_lists = merge_or(
        [index.get(token, []) for token in query_processed]
    )
    
    return [
        Document(doc_id=result_doc_id, name=documents_dict[result_doc_id]) for result_doc_id in merged_posting_lists
    ][:limit]

In [98]:
search_with_or_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное",
    10,
)

[Document(doc_id=31413018, name='Johnson\'s baby Крем детский для лица, тела и рук "3 в 1", с молоком, 50 г'),
 Document(doc_id=33705241, name='Кофе в капсулах Tassimo Jacobs Latte Macchiato Caramel, с жидким молоком, 8 порций'),
 Document(doc_id=135456706, name='Набор контейнеров c герметичными крышками для грудного молока Philips Avent SCF618/10 180 мл, 10 шт'),
 Document(doc_id=135789425, name='Батончик злаковый Corny Milk, с молоком и медом, 30 г'),
 Document(doc_id=135789426, name='Батончик злаковый Corny Milk, с молоком и какао, 30 г'),
 Document(doc_id=136780595, name='Медвежонок Барни Пирожное с молоком, 150 г'),
 Document(doc_id=136780600, name='Медвежонок Барни Пирожное с молоком, 24 шт по 30 г'),
 Document(doc_id=137673490, name='Каша быстрого приготовления Быстров овсяная с клубникой и молоком, 240 г'),
 Document(doc_id=137673491, name='Каша быстрого приготовления Быстров овсяная Ассорти клубника, малина, лесные ягоды с молоком, 6 пакетиков по 40 г, 240 г'),
 Document(doc_i

In [99]:
search_with_or_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное 1литр",
    10,
)

[Document(doc_id=31413018, name='Johnson\'s baby Крем детский для лица, тела и рук "3 в 1", с молоком, 50 г'),
 Document(doc_id=33705241, name='Кофе в капсулах Tassimo Jacobs Latte Macchiato Caramel, с жидким молоком, 8 порций'),
 Document(doc_id=135456706, name='Набор контейнеров c герметичными крышками для грудного молока Philips Avent SCF618/10 180 мл, 10 шт'),
 Document(doc_id=135789425, name='Батончик злаковый Corny Milk, с молоком и медом, 30 г'),
 Document(doc_id=135789426, name='Батончик злаковый Corny Milk, с молоком и какао, 30 г'),
 Document(doc_id=136780595, name='Медвежонок Барни Пирожное с молоком, 150 г'),
 Document(doc_id=136780600, name='Медвежонок Барни Пирожное с молоком, 24 шт по 30 г'),
 Document(doc_id=137673490, name='Каша быстрого приготовления Быстров овсяная с клубникой и молоком, 240 г'),
 Document(doc_id=137673491, name='Каша быстрого приготовления Быстров овсяная Ассорти клубника, малина, лесные ягоды с молоком, 6 пакетиков по 40 г, 240 г'),
 Document(doc_i

In [100]:
search_with_or_condition(
    documents_index,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "диван кровать",
    10,
)

[Document(doc_id=7014376, name='Sylvanian Families Игровой набор Трехъярусная кровать'),
 Document(doc_id=28876761, name='Держатель пялец для вышивания на диване, кресле или за столом, Hobby Nurge'),
 Document(doc_id=149462131, name='Intex Кровать надувная 250х300 см'),
 Document(doc_id=169810294, name='Наматрасник 160х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 Document(doc_id=169810305, name='Наматрасник 80х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 Document(doc_id=169810314, name='Наматрасник 90х200 непромокаемый с резинками по углам, простыня водонепроницаемая непромокаемая, наматрацник водонепроницаемый, чехол на матрас или диван, наматрасник защитный махровый, топпер.'),
 Document(doc_id=170627411, name='Н

In [101]:
calculate_validation_metrics(
    partial(
        search_with_or_condition,
        documents_index,
        documents_dict,
        symbols_to_replace,
        russian_stopwords,
        morph,
    ),
    limit=20,
)

precision = 0.06488461538461539
recall = 0.045055578734379986
f1_score = 0.04776700492730376

## 3.1 Введение в ранжирование

In [102]:
def create_index_with_tf(documents_processed: list[list[str]], limit: Optional[int] = None):
    sample_documents_processed = documents_processed[:limit]
    corpus_tokens = [
        (token, doc_id) for doc_id, doc_tokens in sample_documents_processed for token in doc_tokens
    ]
    corpus_tokens.sort()
    
    index = {}

    for token, doc_id in corpus_tokens:
        if token in index:
            if doc_id == index[token][-1][0]:
                index[token][-1][1] += 1
                continue
            index[token].append([doc_id, 1])
        else:
            index[token] = [[doc_id, 1]]
    
    return index


In [103]:
index_with_tf = create_index_with_tf(documents_processed)

In [105]:
index_with_tf["диван"]

[[28876761, 1],
 [169810294, 1],
 [169810305, 1],
 [169810314, 1],
 [170627411, 1],
 [170627453, 1],
 [170627456, 1],
 [172686265, 1],
 [172686268, 1],
 [172686269, 1],
 [172686270, 1],
 [174148448, 1],
 [174443280, 1],
 [174443281, 1],
 [174443283, 1],
 [177310072, 2],
 [203595521, 1],
 [203598178, 1],
 [204416930, 1],
 [229424677, 1],
 [229425205, 1],
 [245377779, 1],
 [255343578, 1],
 [258876942, 1],
 [258901916, 1],
 [258908855, 1],
 [259150828, 1],
 [262376994, 1],
 [270393871, 1],
 [279745854, 1],
 [282182889, 1],
 [282235399, 1],
 [282764375, 1],
 [302585365, 1],
 [311565940, 1],
 [325179712, 1],
 [339324277, 1],
 [344386617, 2],
 [361843781, 1],
 [367354212, 1],
 [367555536, 1],
 [367575426, 1],
 [367579588, 1],
 [383425290, 1],
 [414307639, 1],
 [418312504, 1],
 [421452489, 1],
 [424884701, 1],
 [478237691, 1],
 [495117560, 2],
 [496408166, 2],
 [509211202, 1],
 [514356916, 1],
 [522969042, 1],
 [523819944, 1],
 [525483453, 1],
 [541335282, 1],
 [546303575, 1],
 [563247577, 1]

In [104]:
index_with_tf["молоко"]

[[31413018, 1],
 [33705241, 1],
 [135456706, 1],
 [135789425, 1],
 [135789426, 1],
 [136780595, 1],
 [136780600, 1],
 [137673490, 1],
 [137673491, 1],
 [137673492, 1],
 [138133907, 1],
 [140463353, 1],
 [140475265, 1],
 [140475273, 1],
 [140675230, 1],
 [140675231, 1],
 [140675232, 1],
 [140675233, 1],
 [140675234, 1],
 [140676398, 1],
 [140676409, 1],
 [140676410, 1],
 [140676411, 1],
 [140676412, 1],
 [140987734, 1],
 [140987736, 1],
 [140987737, 1],
 [140987738, 1],
 [140987742, 1],
 [140987743, 1],
 [141133760, 2],
 [141165025, 1],
 [141165031, 1],
 [141165033, 1],
 [141270536, 1],
 [141270537, 1],
 [141544350, 1],
 [141580513, 1],
 [141580514, 1],
 [141580515, 1],
 [141580516, 1],
 [141580518, 1],
 [141580519, 1],
 [141580520, 1],
 [141580524, 1],
 [141876199, 1],
 [141876200, 1],
 [141876201, 1],
 [141927409, 1],
 [142120587, 1],
 [142120588, 1],
 [142120589, 1],
 [142120590, 1],
 [142479173, 1],
 [142624198, 1],
 [142731191, 1],
 [142731192, 1],
 [143321964, 1],
 [143321967, 1],

In [106]:
with open("documents_index_with_tf.json", "w") as f:
    json.dump(index_with_tf, f)


In [107]:
with open("documents_index_with_tf.json", "r") as f:
    documents_index_with_tf = json.load(f)

### Взвешенный поиск по индексу

In [108]:
def search_with_tf_with_or_condition(
    index: dict[str, list[int]], 
    documents_dict: dict[int, str],
    symbols_to_replace: dict[str, str],
    russian_stopwords: set[str],
    lemmatizer: MorphAnalyzer, 
    query: str, 
    limit: Optional[int] = None,
) -> list[str]:
    query_processed = process_document(query, symbols_to_replace, russian_stopwords, lemmatizer)
    if not query_processed:
        return []
    results = dict()
    for token in query_processed:
        token_results = index.get(token, [])
        for token_result in token_results:
            doc_id, term_freq = token_result[0], token_result[1]
            if doc_id in results:
                results[doc_id] += term_freq
            else:
                results[doc_id] = term_freq
    
    ranked_result_list = sorted(
        [Document(doc_id=result_doc_id, name=[documents_dict[result_doc_id], score]) for result_doc_id, score in results.items()],
        key=lambda x: -x.name[1]
    )
    
    return ranked_result_list[:limit]

In [109]:
search_with_tf_with_or_condition(
    documents_index_with_tf,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "диван кровать",
    10,
)

[Document(doc_id=177310072, name=['Плед Павлайн Ирен какао 150х200 , плед на кровать; плед на диван; покрывало на кровать; покрывало на диван; плед 150 на 200; плед 200 на 220; 150*200; плед 200х220; плед 150х200; полуторка', 4]),
 Document(doc_id=496408166, name=['Покрывало на кровать и диван 117х200 см стеганое, плед для дивана детской и спальни в подарок, бархат, серое АМИ МЕБЕЛЬ Беларусь', 3]),
 Document(doc_id=567905258, name=['Покрывало на кровать и диван 140х220 см стеганое 1,5 спальное, плед для дивана детской и спальни в подарок, бархат, АМИ МЕБЕЛЬ Беларусь', 3]),
 Document(doc_id=627844666, name=['Покрывало плед шерстяной 200х220 см PLEDICO "Скандинавский стиль" на кровать диван евро, серый белый, накидка на диван шерсть, подарочная упаковка, производство Россия', 3]),
 Document(doc_id=655020385, name=['Плед 200х220 на кровать LUZIA, покрывало на кровать, для дивана, двухсторонний, 60% хлопок, бежевый', 3]),
 Document(doc_id=760196461, name=['Плед 200х220 для дивана LAVAL, по

In [110]:
search_with_tf_with_or_condition(
    documents_index_with_tf,
    documents_dict,
    symbols_to_replace,
    russian_stopwords,
    morph,
    "молоко ультрапастеризованное 1литр",
    10,
)

[Document(doc_id=1615880294, name=['Молоко ультрапастеризованное Просто Молоко, 3,2 %, 1 л', 3]),
 Document(doc_id=1615880546, name=['Молоко ультрапастеризованное Просто Молоко, 2,5 %, 1 л', 3]),
 Document(doc_id=140987734, name=['Valio Молоко Ультрапастеризованное 1.5% 250мл. 1шт.', 2]),
 Document(doc_id=140987737, name=['Valio Молоко Ультрапастеризованное 2.5% 1000мл. 1шт.', 2]),
 Document(doc_id=140987742, name=['Valio Молоко Ультрапастеризованное 3.5% 1000мл. 1шт.', 2]),
 Document(doc_id=141133760, name=['Шоколад Ritter Sport "Альпийское молоко" с альпийским молоком, 100 г', 2]),
 Document(doc_id=141165025, name=['Молоко ультрапастеризованное Простоквашино, 3,2%, 950 мл', 2]),
 Document(doc_id=141270536, name=['Молоко Вкуснотеево ультрапастеризованное, 2,5%, 950 мл', 2]),
 Document(doc_id=141270537, name=['Молоко ультрапастеризованное, 3,2%, 950 мл, Вкуснотеево', 2]),
 Document(doc_id=141580513, name=['Молоко ультрапастеризованное 3,5%, 1 л, Parmalat', 2])]

In [111]:
calculate_validation_metrics(
    partial(
        search_with_tf_with_or_condition,
        documents_index_with_tf,
        documents_dict,
        symbols_to_replace,
        russian_stopwords,
        morph,
    ),
    limit=20,
)

precision = 0.1493846153846154
recall = 0.13514921457410067
f1_score = 0.1283941490170208